In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error

from sklearn import ensemble
from sklearn.inspection import permutation_importance

In [2]:
df = pd.read_csv('hotData.csv')


In [3]:
df.head()

,goals,xg,crosses,boxtouches,passes,progpasses,takeons,progruns,tackles,interceptions,...,64,65,66,67,68,69,70,71,72,73
0,0.055385,-0.119231,-0.020769,0.236154,-0.781538,-0.649231,-0.407692,-0.568462,-0.578462,-0.444615,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,-0.243077,0.063846,-0.090769,-0.287692,-0.704615,-0.441538,-0.098462,-0.383077,-0.722308,-0.606154,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,-0.420000,-0.230000,-0.150769,-0.435385,-0.687692,-0.408462,-0.256923,-0.586154,-0.629231,-0.560000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,-0.420000,-0.230000,-0.150769,-0.435385,-0.687692,-0.408462,-0.256923,-0.586154,-0.629231,-0.560000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,-0.070769,-0.368462,-0.493846,-0.247692,-0.681538,-0.536923,-0.210000,-0.427692,-0.539231,-0.622308,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [4]:
df.columns

Index(['goals', 'xg', 'crosses', 'boxtouches', 'passes', 'progpasses',
       'takeons', 'progruns', 'tackles', 'interceptions', 'clearances',
       'blocks', 'aerials', 'fouls', 'fouled', 'nsxg', 'results', '0', '1',
       '2', '3', '4', '5', '6', '7', '8', '9', '10', '11', '12', '13', '14',
       '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26',
       '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38',
       '39', '40', '41', '42', '43', '44', '45', '46', '47', '48', '49', '50',
       '51', '52', '53', '54', '55', '56', '57', '58', '59', '60', '61', '62',
       '63', '64', '65', '66', '67', '68', '69', '70', '71', '72', '73'],
      dtype='object')

In [18]:
y = df['results']
X = df.drop(columns=['results'])

X_train, X_test, y_train, y_test = train_test_split(
     X, y, test_size=0.10, random_state=2)

In [19]:
params = {
    "n_estimators": 3990,
    "max_depth": 400,
    "min_samples_split":4,
    "learning_rate": 0.0159,
    "loss": "squared_error",
    "max_features": 16,
    "min_weight_fraction_leaf": 0.11,
#     "subsample": 0.8
    "min_impurity_decrease": 5,
    
}

In [20]:
reg = ensemble.GradientBoostingRegressor(**params)
reg.fit(X_train, y_train)

print('GRADIENT BOOSTING REGRESSION')

test_pred = reg.predict(X_test)

print('RMSE: '+str(np.sqrt(mean_squared_error(y_test, test_pred))))
print('R Squared: '+str(r2_score(y_test, test_pred)))
print('MSE TEST: '+str(mean_squared_error(y_test, test_pred)))
print('MAE TEST: '+str(mean_absolute_error(y_test, test_pred)))
print('MAPE: '+str(mean_absolute_percentage_error(y_test, test_pred)))


GRADIENT BOOSTING REGRESSION
RMSE: 7.034873682497054
R Squared: 0.3786668029166095
MSE TEST: 49.48944772868966
MAE TEST: 5.434607816190279
MAPE: 0.7709625322254346


In [ ]:
# MAPE: 0.7709625322254346
    

In [21]:
# mape = 1.0765
# mape = 1.0647916900630887 max_feat = 16 
# mape 1.059828673697475 / estimators 5000
# mape 1.0597337473020356 / estimators 4000
# mape 1.0597326684323638 / estimators 4990
# mape 1.0432022275261308 / min fraction leaf .12
#mape 1.031433617032885 / '' ''     ''   '' .11
# ** Results vary -mape .9879763523310932 / "subsample": 0.8
# mpae  1.0241404892305128 / min_impurity_decrease 10
# mape 1.0097789777918718 / min decrease 5

In [22]:
data = {'y_test': y_test, 'y_pred': test_pred}

In [23]:
new_df = pd.DataFrame(data)

In [24]:
diff = (new_df['y_test'] - new_df['y_pred']).abs()

In [25]:
new_df['diff'] = diff
new_df = new_df.sort_values(by='diff', ascending=True)
new_df['y_pred'] = round(new_df['y_pred'], 1)
new_df['rounded'] = round(new_df['y_pred'])
new_df['new_diff'] = (new_df['y_test'] - new_df['rounded']).abs()

In [26]:
new_df

,y_test,y_pred,diff,rounded,new_diff
144,4,3.8,0.151487,4.0,0.0
11,12,11.8,0.228059,12.0,0.0
66,17,17.3,0.325928,17.0,0.0
130,10,10.6,0.612782,11.0,1.0
141,14,14.6,0.619070,15.0,1.0
106,9,9.9,0.928963,10.0,1.0
151,22,20.3,1.670596,20.0,2.0
29,23,20.8,2.167599,21.0,2.0
7,19,22.4,3.446106,22.0,3.0
244,6,9.5,3.538567,10.0,4.0


In [27]:
new_df['diff'].mean()

5.434607816190279

In [28]:
new_df['diff'].median()

4.774921612382311

In [29]:
new_df['new_diff'].mean()


5.413793103448276

In [30]:
new_df['new_diff'].median()

5.0

In [18]:
# from sklearn.linear_model import Ridge
# from sklearn.feature_selection import SequentialFeatureSelector
# from sklearn.model_selection import TimeSeriesSplit

# rr = Ridge(alpha=1)

# split = TimeSeriesSplit(n_split=3)

# sfs = SequentialFeatureSelector(rr, features_to_select=17, 
#                                 direction="forward", cv=split, 
#                                 n_jobs=4)


In [19]:
df.shape

(282, 91)